# PEFT 库 QLoRA 实战 - ChatGLM3-6B

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [14]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
train_data_path = 'HasturOfficial/adgen'    # 训练数据路径
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 4                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
resume_from_checkpoint = None             # 如果从checkpoint恢复训练，指定路径
prompt_text = ''                          # 所有数据前的指令文本
compute_dtype = 'fp32'                    # 计算数据类型（fp32, fp16, bf16）

# 数据准备

In [15]:
from datasets import load_dataset

dataset = load_dataset(train_data_path)

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 114599
    })
    validation: Dataset({
        features: ['content', 'summary'],
        num_rows: 1070
    })
})

In [17]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [18]:
show_random_elements(dataset["train"], num_examples=20)


,content,summary
0,类型#上衣*材质#涤纶*材质#纤维*图案#线条*衣样式#风衣*衣样式#外套*衣门襟#无扣,优选客供莱<UNK>纤维+涤纶材质，手感亲肤细腻，表面别致的立体压褶提花，清晰有质感。设计感时尚的风衣外套，采用无扣开门襟设计，线条优美，展现随意个性又能凹造型。藏于绗缝的两侧插袋，实用又方便。
1,类型#上衣*颜色#红色*风格#复古*风格#中国风*图案#复古*图案#刺绣*衣样式#马甲*衣样式#外套*衣款式#盘扣,马甲外套是早春的时尚产物，能够对坑微凉的气候，又能带出属于时尚的搭配功能，将实用性与美观性并兼。复古的大红色，增添女性的柔美气息。盘扣装饰的门襟，展现出古典雅致的中国风。惟妙惟肖的刺绣图案，为裙身带出一笔唯美的即视感。侧边开衩的处理，行走间绽放出荷尔蒙的吸引力。
2,类型#上衣*版型#宽松*风格#英伦*图案#格子*衣样式#马甲*衣样式#西装*衣长#短款,换季的时尚怎么能少的<UNK>一款气质的西装马甲做点缀呢？干练的短款版型与宽松的设计想邂逅，散发出bf风的帅气之感。衣身格纹的点缀更是有一种英伦格调的绅士感呢。
3,类型#上衣*版型#显瘦*风格#知性*风格#休闲*衣样式#衬衫*衣款式#腰带,高腰的裤型，修饰出修长的腿型，显瘦的同时还能轻松遮住各种赘肉；裤子后袋设计，增加几分休闲时髦感；上衣的衬衫样式，帅气知性，很有女人味。腰部配了腰带设计，轻松细腰，显瘦又拉长比例，非常有格调。
4,类型#裙*图案#印花*裙型#a字*裙下摆#荷叶边*裙袖长#七分袖*裙领型#v领*裙领型#翻领,大翻领加荷叶边的设计非常甜美，v领的设计也凸显出精致的锁骨。<UNK>印花的图案很可爱，排列也非常有设计感。a字版型有娇憨的感觉，也非常遮肉。恰到好处的裙长让腿部更加纤细修长。七分袖的设计也露出了手臂最细的部分。
5,类型#裙*风格#复古*风格#性感*图案#格子*图案#复古*图案#线条*裙长#连衣裙*裙领型#v领*裙款式#拼接*裙款式#绑带,实穿率超强的一件连衣裙，配以插画格子图案，丰富裙身很是独特。复古的v领视觉上修饰颈部线条，配以可调节的肩带增添一份小性感。而腰间的绑带设计，无形之中修饰腰部曲线更显小女人的气息。撞色织带拼接的下摆，平添裙身的复古气息！
6,类型#裙*材质#牛仔布*风格#简约*图案#撞色*裙型#牛仔裙*裙型#包臀裙*裙型#鱼尾裙*裙长#半身裙*裙衣长#中长款,设计剪裁立体有心意，穿着更加有型有范儿。设计师匠心独制，中长款设计为此款包臀裙添彩，另外牛仔元素融合鱼尾设计又将半身裙点缀得恰到好处。简约大方的牛仔，使每个女孩子都能轻松驾驭，展现夏日里的好身材，释放女性独特魅力。裙身撞色的设计，丰富裙身的层次感。
7,类型#上衣*版型#显瘦*图案#线条*衣样式#打底衫*衣领型#v领*衣袖型#喇叭袖*衣款式#飘带,小巧的飘带领口，优雅又带有小俏皮，微微丝光材质，轻巧的营造出衣物的流动感。宽容的喇叭袖子设计，很好地凸显出纤细的手臂线条，而且不挑风格，优雅俏皮都能驾驭，举手投足间散发优雅气息。圆v领的设计时髦又能突显完美锁骨，修身显瘦的版型怎么穿怎么好看，可外穿也可以作为打底衫。
8,类型#裤*图案#印花*裤款式#口袋*裤腰型#松紧腰*裤口#小脚,这款时尚印花休闲裤采用松紧束脚版型剪裁设计，不仅可以轻松驾驭各种穿搭单品，而且还让穿着者的腿部更加笔直、修长，非常适合人们的日常穿搭；而侧边口袋做工精良，优雅有型，品质保证，穿着有型又不失格调。
9,类型#裙*材质#天丝*风格#简约*风格#高贵*裙型#百褶*裙型#a字*裙长#半身裙*裙款式#松紧带,这款半身裙采用了似烟雾的蓝灰色调，自带满满的仙气感。光滑的天丝面料散发才淡淡的光泽感，尽显简约大气的美。a字的百褶裙设计，让你行走间流露出一股慵懒随性的韵味，让人无法抗拒。松紧带的设计，一秒勾勒出腰身，尽显优雅的高贵女神范。


# 使用 ChatGLM3-6b Tokenizer 处理数据

In [7]:
from transformers import AutoTokenizer

# revision='b098244' 版本对应的 ChatGLM3-6B 设置 use_reentrant=False
# 最新版本 use_reentrant 被设置为 True，会增加不必要的显存开销
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                         )

In [8]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question,padding="max_length" , add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer,padding="max_length" , add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}

In [9]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)

In [10]:
show_random_elements(tokenized_dataset, num_examples=1)


,input_ids,labels
0,"[64790, 64792, 30910, 33467, 31010, 56778, 30998, 56778, 54888, 31010, 37462, 30910, 36250, 55683, 54848, 32492, 31735, 32144, 39745, 40786, 31759, 41424, 43386, 54892, 54657, 55360, 56300, 54530, 37027, 31123, 41099, 33109, 34002, 31835, 33168, 51827, 32291, 41553, 56059, 54661, 54619, 35878, 31155, 37462, 56778, 57069, 55426, 31735, 33612, 41256, 31123, 54377, 54617, 56164, 33168, 33588, 32998, 32311, 34219, 46948, 31155, 54882, 54892, 54657, 58747, 55881, 32048, 54986, 54803, 31123, 55432, 54557, 32647, 36803, 54530, 35878, 48465, 31155, 2]","[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 30910, 36250, 55683, 54848, 32492, 31735, 32144, 39745, 40786, 31759, 41424, 43386, 54892, 54657, 55360, 56300, 54530, 37027, 31123, 41099, 33109, 34002, 31835, 33168, 51827, 32291, 41553, 56059, 54661, 54619, 35878, 31155, 37462, 56778, 57069, 55426, 31735, 33612, 41256, 31123, 54377, 54617, 56164, 33168, 33588, 32998, 32311, 34219, 46948, 31155, 54882, 54892, 54657, 58747, 55881, 32048, 54986, 54803, 31123, 55432, 54557, 32647, 36803, 54530, 35878, 48465, 31155, 2]"


# 数据集处理：shuffle & flatten

In [11]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)


In [12]:
tokenized_dataset = tokenized_dataset.flatten_indices()



# 定义 DataCollatorForChatGLM 类 批量处理数据

In [13]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}

In [14]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

# 训练模型

## 加载 ChatGLM3-6B 量化模型
## 使用 nf4 量化数据类型加载模型，开启双量化配置，以bf16混合精度训练，预估显存占用接近4GB

In [15]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

In [16]:
# 加载模型

In [17]:
# revision='b098244' 版本对应的 ChatGLM3-6B 设置 use_reentrant=False
# 最新版本 use_reentrant 被设置为 True，会增加不必要的显存开销
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
# 获取当前模型占用的 GPU显存（差值为预留给 PyTorch 的显存）
memory_footprint_bytes = model.get_memory_footprint()
memory_footprint_mib = memory_footprint_bytes / (1024 ** 2)  # 转换为 MiB

print(f"{memory_footprint_mib:.2f}MiB")

3739.69MiB


# 预处理量化模型

In [19]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training

kbit_model = prepare_model_for_kbit_training(model)

# 自定义模型新增 Adapter

In [20]:
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

In [21]:
target_modules

['query_key_value']

In [22]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [23]:
qlora_model = get_peft_model(kbit_model, lora_config)


In [24]:
qlora_model.print_trainable_parameters()


trainable params: 974,848 || all params: 6,244,558,848 || trainable%: 0.0156


# 训练超参数配置

1个epoch表示对训练集的所有样本进行一次完整的训练。
- num_train_epochs 表示要完整进行多少个 epochs 的训练。
- 关于使用 num_train_epochs 时，训练总步数 steps 的计算方法
- 训练总步数： total_steps = steps/epoch * num_train_epochs
- 每个epoch的训练步数：steps/epoch = num_train_examples / (batch_size * gradient_accumulation_steps)
以 adgen 数据集为例计算
```
DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 114599
    })
    validation: Dataset({
        features: ['content', 'summary'],
        num_rows: 1070
    })
})
```
代入超参数和配置进行计算：
```
num_train_epochs = 1
num_train_examples = 114599
batch_size = 16
gradient_accumulation_steps = 4


steps = num_train_epochs * num_train_examples / (batch_size * gradient_accumulation_steps)
      = 1 * 114599 / (16 * 4)
      = 1790
```

In [25]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=f"models/{model_name_or_path}",          # 输出目录
    per_device_train_batch_size=16,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=4,                     # 梯度累积步数
    # per_device_eval_batch_size=8,                      # 每个设备的评估批量大小
    learning_rate=1e-3,                                # 学习率
    num_train_epochs=1,                                # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.1,                                  # 预热比例
    logging_steps=10,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=100,                                    # 模型保存步数
    # evaluation_strategy="steps",                       # 评估策略
    # eval_steps=500,                                    # 评估步数
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)


In [26]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [33]:
from transformers import TrainingArguments, Trainer

training_demo_args = TrainingArguments(
    output_dir=f"../model/demo/{model_name_or_path}",          # 输出目录
    per_device_train_batch_size=2,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=4,                     # 梯度累积步数
    learning_rate=1e-3,                                # 学习率
    max_steps=100,                                     # 训练步数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.1,                                  # 预热比例
    logging_steps=10,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=20,                                    # 模型保存步数
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)

In [34]:
trainer = Trainer(
        model=qlora_model,
        args=training_demo_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [35]:
trainer.train()


Step,Training Loss
10,4.681600
20,3.854900
30,3.727300
40,3.711400
50,3.662000
60,3.531000
70,3.588000
80,3.552000
90,3.589700
100,3.556500


TrainOutput(global_step=100, training_loss=3.745430145263672, metrics={'train_runtime': 82.585, 'train_samples_per_second': 9.687, 'train_steps_per_second': 1.211, 'total_flos': 3777398432931840.0, 'train_loss': 3.745430145263672, 'epoch': 0.006980802792321117})

In [36]:
trainer.model.save_pretrained(f"../model/demo/{model_name_or_path}")

# 模型推理 - 使用 QLoRA 微调后的 ChatGLM3-6B

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
peft_model_path = f"../model/demo/{model_name_or_path}"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [2]:
config = PeftConfig.from_pretrained(peft_model_path)

q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=torch.float32)

base_model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                       quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto')
base_model.requires_grad_(False)
base_model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [3]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
peft_model = PeftModel.from_pretrained(base_model, peft_model_path)


Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


# 微调前后效果对比

In [4]:
def base_model_interface(input_text:str)->any:
    response, history = base_model.chat(tokenizer=tokenizer, query=input_text)
    return response

def lora_model_interface(input_text:str)->any:
    response, history = peft_model.chat(tokenizer=tokenizer, query=input_text)
    return response

# 对比 例子 1

In [8]:
input_text = '类型#裙*版型#显瘦*风格#文艺*风格#简约*图案#印花*图案#撞色*裙下摆#压褶*裙长#连衣裙*裙领型#圆领'
print(f'输入：\n{input_text}')
print(f"ChatGLM3-6B 微调前: \n{base_model_interface(input_text)}")
print(f"ChatGLM3-6B 微调后: \n{lora_model_interface(input_text)}")

输入：
类型#裙*版型#显瘦*风格#文艺*风格#简约*图案#印花*图案#撞色*裙下摆#压褶*裙长#连衣裙*裙领型#圆领
ChatGLM3-6B 微调前: 
这款连衣裙采用了撞色和印花的设计，显得非常有文艺的感觉，简单大气，又带有一丝优雅的气息。圆领的设计，显得优雅又温柔，更有一种温婉的气质。裙摆采用了经典的A字型设计，简单又显瘦，又显得优雅又大方。
ChatGLM3-6B 微调后: 
这款连衣裙采用经典的圆领设计，凸显出颈部的线条美感，展现女性优雅气质。整体设计采用撞色拼接，彰显出视觉的层次感。简约的图案，展现出文艺气息，彰显出时尚大气的感觉。裙摆采用休闲的压褶设计，不仅显瘦还展现出简约大方的气质。


# 对比 例子 2

In [9]:
input_text = '类型#上衣*版型#宽松*风格#英伦*图案#格子*衣样式#马甲*衣样式#西装*衣长#短款'
print(f'输入：\n{input_text}')
print(f"ChatGLM3-6B 微调前: \n{base_model_interface(input_text)}")
print(f"ChatGLM3-6B 微调后: \n{lora_model_interface(input_text)}")

输入：
类型#上衣*版型#宽松*风格#英伦*图案#格子*衣样式#马甲*衣样式#西装*衣长#短款
ChatGLM3-6B 微调前: 
这款短款西装外套采用英伦风设计，宽松的版型设计，穿着舒适又自然。上下采用了经典的格纹设计，充满时尚感。马甲设计，增加层次感，让整体造型更有层次感。
ChatGLM3-6B 微调后: 
宽松的短款西装外套，用经典的格纹元素来装饰，显得更有英伦风的味道。衣身采用了宽松的版型，凸显了随性的气质，同时也不失时尚感。衣袖处采用了双排扣设计，让穿着更具有层次感。衣身采用了马甲来装饰，更是增添了几分休闲的气息。


# 对比 例子 3

In [10]:
input_text = '类型#上衣*版型#显瘦*风格#知性*风格#休闲*衣样式#衬衫*衣款式#腰带'
print(f'输入：\n{input_text}')
print(f"ChatGLM3-6B 微调前: \n{base_model_interface(input_text)}")
print(f"ChatGLM3-6B 微调后: \n{lora_model_interface(input_text)}")

输入：
类型#上衣*版型#显瘦*风格#知性*风格#休闲*衣样式#衬衫*衣款式#腰带
ChatGLM3-6B 微调前: 
衬衫领带，可以轻松搭配出知性休闲风格，还可以修饰腰部线条，显瘦又显高挑。在衬衫的腰带处，还采用了双排扣设计，在视觉上突出腰线，还能让身材更加修长。
ChatGLM3-6B 微调后: 
衬衫领的衬衫，是很多女性朋友的最爱。这款衬衫的版型流畅，可以很好地修饰身材，并且能够很好地展现女性的气质。领口是经典的衬衫领，显得知性大气，十分优雅。衣身是宽松的版型，能够很好地修饰身材，同时穿着十分舒适，十分休闲。腰间有系腰带，可以很好的凸显腰身，十分显瘦。


# 对比 例子 4

In [ ]:
input_text = '类型#裙*图案#印花*裙型#a字*裙下摆#荷叶边*裙袖长#七分袖*裙领型#v领*裙领型#翻领'
print(f'输入：\n{input_text}')
print(f"ChatGLM3-6B 微调前: \n{base_model_interface(input_text)}")
print(f"ChatGLM3-6B 微调后: \n{lora_model_interface(input_text)}")

In [ ]:
# 对比 例子 5

In [ ]:
input_text = '类型#上衣*版型#显瘦*风格#知性*风格#休闲*衣样式#衬衫*衣款式#腰带'
print(f'输入：\n{input_text}')
print(f"ChatGLM3-6B 微调前: \n{base_model_interface(input_text)}")
print(f"ChatGLM3-6B 微调后: \n{lora_model_interface(input_text)}")

In [ ]:
# 对比 例子 6

In [ ]:
input_text = '类型#上衣*版型#显瘦*风格#知性*风格#休闲*衣样式#衬衫*衣款式#腰带'
print(f'输入：\n{input_text}')
print(f"ChatGLM3-6B 微调前: \n{base_model_interface(input_text)}")
print(f"ChatGLM3-6B 微调后: \n{lora_model_interface(input_text)}")

In [ ]:
# 对比 例子 7

In [ ]:
input_text = '类型#上衣*版型#显瘦*风格#知性*风格#休闲*衣样式#衬衫*衣款式#腰带'
print(f'输入：\n{input_text}')
print(f"ChatGLM3-6B 微调前: \n{base_model_interface(input_text)}")
print(f"ChatGLM3-6B 微调后: \n{lora_model_interface(input_text)}")

In [ ]:
# 对比 例子 8

In [ ]:
input_text = '类型#上衣*版型#显瘦*风格#知性*风格#休闲*衣样式#衬衫*衣款式#腰带'
print(f'输入：\n{input_text}')
print(f"ChatGLM3-6B 微调前: \n{base_model_interface(input_text)}")
print(f"ChatGLM3-6B 微调后: \n{lora_model_interface(input_text)}")

In [ ]:
# 对比 例子 9

In [ ]:
input_text = '类型#上衣*版型#显瘦*风格#知性*风格#休闲*衣样式#衬衫*衣款式#腰带'
print(f'输入：\n{input_text}')
print(f"ChatGLM3-6B 微调前: \n{base_model_interface(input_text)}")
print(f"ChatGLM3-6B 微调后: \n{lora_model_interface(input_text)}")

In [ ]:
# 对比 例子 10

In [ ]:
input_text = '类型#上衣*版型#显瘦*风格#知性*风格#休闲*衣样式#衬衫*衣款式#腰带'
print(f'输入：\n{input_text}')
print(f"ChatGLM3-6B 微调前: \n{base_model_interface(input_text)}")
print(f"ChatGLM3-6B 微调后: \n{lora_model_interface(input_text)}")